Import potrzebnych bibliotek

In [2]:
import pandas as pd
import numpy as np
import scipy as sp

# Zadanie 1

Załóżmy, że funkcja stopy terminowej miała postać r(t) = 0,01+0,005t, gdy dziesięcioletnia obligacja zerokuponowa została zakupiona. Pół roku później, gdy sprzedano
obligację, funkcja stopy terminowej miała postać r(t) = 0,02 + 0,001t. Jaka była
stopa zwrotu z tej inwestycji?

In [14]:
def f1(t) ->float:
    return 0.01+0.005*t
def f2(t) ->float:
    return 0.02+0.001*t
I1,err1 = sp.integrate.quad(f1,0,10)
I2,err2 = sp.integrate.quad(f2,0,9.5)

P1 = np.e **(-I1)
P2 = np.e **(-I2)
print("P1 =",P1)
print("P2 =",P2)

R =  P2/P1 - 1

print("R =",R)

P1 = 0.7046880897187134
P2 = 0.7904720344486094
R = 0.1217332121565129


# Zadanie 2

 Załączony plik TermStructureZero.csv zawiera symulowane dane 
 dotyczące 20 obligacji zerokuponowych. 
 W kolejnych polach znajdują się:

 Maturity  = liczba lat do wykupu,
 Quote     = kurs obligacji.   

 Napisz kod w R, który uzupełni plik o pola:

 SpotRate    = stopa spot dla n lat (zerokuponowa)
 ForwardRate = stopa terminowa dla n-tego roku.

In [13]:
df = pd.read_csv('TermStructureZero.csv')
df["SpotRate"] = ((100 / df["Quote"])\
                    **(1 / df["Maturity"])\
                        -1)
                     
df['ForwardRate'] = ((1+df["SpotRate"])**(df.index+1))/((1+df["SpotRate"].shift(1))**(df.index))-1

print(df.head(20))

    Maturity  Quote  SpotRate  ForwardRate
0          1  99.01  0.009999     0.009999
1          2  98.70  0.006564     0.003141
2          3  98.36  0.005527     0.003457
3          4  97.64  0.005989     0.007374
4          5  97.01  0.006090     0.006494
5          6  96.11  0.006635     0.009364
6          7  95.13  0.007158     0.010302
7          8  93.89  0.007912     0.013207
8          9  92.58  0.008603     0.014150
9         10  91.02  0.009453     0.017139
10        11  89.35  0.010290     0.018691
11        12  87.48  0.011209     0.021376
12        13  85.45  0.012169     0.023757
13        14  83.42  0.013033     0.024335
14        15  81.04  0.014114     0.029368
15        16  78.54  0.015212     0.031831
16        17  75.89  0.016361     0.034919
17        18  73.11  0.017553     0.038025
18        19  70.14  0.018843     0.042344
19        20  67.03  0.020203     0.046397


# Zadanie 3

Załączony plik TermStructure.csv zawiera symulowane dane 
dotyczące 20 obligacji o kuponach wypłacanych raz w roku 
(pierwsza wypłata za rok). W kolejnych polach znajdują się:

Maturity  = liczba lat do wykupu,
FV        = wartość nominalna,   
Coupon    = wartość kuponu,
Price     = cena obligacji,

Napisz kod w R, który uzupełni plik o pola:

SpotRate      = stopa spot dla n lat (zerokuponowa)
ForwardRate   = stopa terminowa dla n-tego roku.

# Zadanie 4

 Dla notowań spółek giełdowych A,B,C z pliku NotowaniaABC.csv wyznacz

 a) oczekiwane zwroty i odchylenia standardowe tych zwrotów dla każdej 
    ze spółek A,B,C;
 b) oczekiwany zwrot i odchylenie standardowe zwrotów dla portfela z wagami 

     w_A=0.2,  w_B=0.3,  w_C=0.5.

In [43]:
df = pd.read_csv('NotowaniaABC.csv')
print(df.head())

# Oczekiwane zwroty
SA = np.mean((df['A']-df['A'].shift(1))/df['A'].shift(1))
SB = np.mean((df['B']-df['B'].shift(1))/df['B'].shift(1))
SC = np.mean((df['C']-df['C'].shift(1))/df['C'].shift(1))

# Odchylenia standardowe zwrotów
DA = np.std((df['A']-df['A'].shift(1))/df['A'].shift(1), ddof = 1)
DB = np.std((df['B']-df['B'].shift(1))/df['B'].shift(1), ddof = 1)
DC = np.std((df['C']-df['C'].shift(1))/df['C'].shift(1), ddof = 1)

print(f"Oczekiwane zwroty:{SA} {SB} {SC}")
print(f"Wariancje zwrotow :{DA**2} {DB**2} {DC**2}")

        A       B       C
0  35.191  32.696  17.288
1  37.096  31.281  15.100
2  36.409  32.318  15.021
3  36.631  37.609  16.351
4  43.442  37.960  16.460
Oczekiwane zwroty:0.06666585167772418 0.022770019526007032 0.13105739580118375
Wariancje zwrotow :0.01970113991639636 0.018037434053438203 0.04300318271253653


In [45]:

w = np.array([0.2, 0.3, 0.5])


returns = df[['A','B','C']].pct_change().dropna()

cov_matrix = returns.cov()

expected_return_wallet = np.dot(w, returns.mean())

std_portfolio_wallet = np.sqrt(np.dot(w, np.dot(cov_matrix, w)))

print(f"Oczekiwany zwrot portfela: {expected_return_wallet:.4f}")
print(f"Odchylenie standardowe portfela: {std_portfolio_wallet:.4f}")

Oczekiwany zwrot portfela: 0.0857
Odchylenie standardowe portfela: 0.1309


# Zadanie 5

 Dla notowań spółek giełdowych A,B,C z pliku NotowaniaABC.csv wyznacz,
 korzystając z odpowiednich twierdzeń z wykładu 4:

a) wagi portfela o minimalnej wariancji; \
b) wagi portfela o minimalnej wariancji i oczekiwanym zwrocie 15%.

In [2]:
import numpy as np
import pandas as pd


df = pd.read_csv("NotowaniaABC.csv")


returns = df[['A','B','C']].pct_change().dropna()
mean_returns = returns.mean()
cov_matrix = returns.cov()


C = cov_matrix.values
C_inv = np.linalg.inv(C)

u = np.ones(3)

A = u @ C_inv @ u.T

wagi = (u @ C_inv) / A

print("a) Wagi portfela minimalnej wariancji :")
print(wagi)


a) Wagi portfela minimalnej wariancji :
[0.35867484 0.47000256 0.17132261]


In [3]:

r = 0.15
mu = mean_returns.values
u = np.ones(3)

B = mu @ C_inv @ u.T
C = mu @ C_inv @ mu.T
D = u @ C_inv @ mu.T
Delta = A*C - B**2

w = (((C - r * B))*(u @ C_inv ) + (A*r - D) * (mu @ C_inv ))/Delta

print(w)

[ 0.73852829 -0.61408433  0.87555604]
